# Explore creation of phenopackets from supplemental material
Let's take Platzer K. De Novo Variants in MAPK8IP3 Cause Intellectual Disability with Variable Brain Anomalies. Am J Hum Genet. 2019 Feb 7;104(2):203-212. PMC6369540.
as an example

In [1]:
import phenopackets as pp
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
import os
import sys

sys.path.insert(0, os.path.abspath('../../pyphetools'))
from pyphetools import CohortEncoder, ColumnMapper, HpoParser, HpoConceptRecognizer

<h2>Importing HPO data</h2>
<p>pyphetools uses the Human Phenotype Ontology (HPO) to encode phenotypic features. The recommended way of doing this is to ingest the hp.json file using HpoParser, which in turn creates an HpoConceptRecognizer object. </p>

In [2]:
hpo_json_path = '/home/peter/data/hpo/hp.json'
parser = HpoParser(hpo_json_file=hpo_json_path)
hpo_cr = parser.get_hpo_concept_recognizer()

Supplemental Table S1 contains Detailed Clinical Information for All Individuals with Causative De Novo Variants in MAPK8IP3. We need to read this from the original Excel file because some of the cells contain new-line symbols.

In [3]:
df = pd.read_excel('data/mmc2.xlsx')

<h2>Stepwise encoding of the supplementary material</h2>
<p>pyphetools supports efficient and accurate HPO-encoding of typical supplementary tables that describe a cohort of 
individuals found to have a certain disease. We recommend using the <tt>df.head()</tt> and the <tt>df.columns</tt> commands
to view the data and the column headers as follows.</p>

In [4]:
df.head()

,Indvidual\nin\nmanuscript,g.(hg19) Chr16:,Transcript\nNM_015133.4\nc.,p.,origin,genetic testing,Sex,age at last assesment,prenatal period,Exam at birth,...,neurological examination,result of external MRI,seizures,Sz onset and Sz types,AEDs used,Sz outcome,EEG,Additional symptoms,family history,further results of genetic testing
0,1,1756405,c.65delG,p.Gly22Alafs*3,de novo,TrioWES,M,14 y 8 m,NaN,41 weeks:\nlength: 53.3 cm\nweight: 3.941 kg\nOFC: NA,...,ataxia,"mild cerebellar atrophy, hypointensity of the globi pallidi and substantia nigra, possible mild degree of abnormal iron or mineral deposition",no,NaN,NaN,NaN,NaN,"speech is ataxic but speaks in sentences/short phrases; attention issues, impulse control and emotional lability, OCD symptoms; recently developed scoliosis",unremarkable,NaN
1,2,1756419,c.79G>T,p.Glu27*,de novo,SingleWES,M,4 y,NaN,length: 49 cm\nweigth: 3215 g\nOFC: 35 cm,...,ataxia,normal,no,NaN,NaN,NaN,NaN,pre-natal pelvi-ureteric junction stenosis (spontaneous resolution at 6 m),NaN,NaN
2,3,1756451,c.111C>G,p.Tyr37*,de novo,TrioWES,M,4 y,NaN,length: 20.5 in\nweight: 8 lb 2 oz\nOFC: NA,...,NaN,Stable areas of T2 hyperintensity involving the central tegmental tracts,no,NaN,NaN,NaN,NaN,Nystagmus,unremarkable,770 kb duplicaion of 20p12.3 on chromosome microarray
3,4,1798706,c.1198G>A,p.Gly400Arg,de novo,TrioWES,M,7 y 6 m,"no prenatal care, no known problems","32 weeks:\nlength: NA,\nweight: 4 lbs,\nOFC: NA\n\nhad a 30 day hospital course",...,NaN,no MRI done,no,NaN,NaN,NaN,NaN,"Left hearing loss; Dysmorphic features: hypertelorism inner canthal distance 4.3cm; low set prominent ears, slight overhangin columella, hypodontia; 5th finger clinodactyly and 5th finger brachydactylky; synophrys; Encopresis",Mother with learning disorder; finished 11th grade; Father with ADHD and learning disorder; finished 9th grade; Full sister with learning disorder; Full sister no known problems; Full brother with learning disorder,NaN
4,5,1810410,c.1331T>C,p.Leu444Pro,de novo,TrioWES,M,10 y,NaN,"40 weeks, length: 52 cm\nweight: 3810 g\nOFC: 36 cm",...,NaN,perisylvian polymicrogyria,yes,10 y:\none event of a generalized seizure,NaN,NaN,"pathological EEG with normal age-related background activity (alpha-type), increased appearance of slowing over temporal and occipital regions","no dysmorphism, small teeth, severe s-configured scoliosis of thoracic and lumbar spine",NaN,NaN


In [5]:
df.columns

Index(['Indvidual\nin\nmanuscript', 'g.(hg19) Chr16:',
       'Transcript\nNM_015133.4\nc.', 'p.', 'origin', 'genetic testing', 'Sex',
       'age at last assesment', 'prenatal period', 'Exam at birth',
       'body measurements\n(at last assesment if not otherwise specified)',
       'DD', 'severity of ID', 'development', 'regression', 'autism',
       'hypotonia', 'movement disorder', 'CVI', 'neurological examination',
       'result of external MRI', 'seizures', 'Sz onset and Sz types',
       'AEDs used', 'Sz outcome', 'EEG', 'Additional symptoms',
       'family history', 'further results of genetic testing'],
      dtype='object')

#We need to create a dictionary with the HPO terms contained in the descriptions

##  ColumnMapper
The idea is to make one ColumnMapper object for each column of interest. The column mapper knows how to map the contents using either
default exact text matching or custom maps from whatever strings to HPO terms.

In [6]:
df['neurological examination'].unique()

array(['ataxia', nan, 'spastic paraplegia',
       'spasticity; nerve conduction and EMG studies with abnormal findings "remarkable for the failure to activate the leg muscles due to an upper motor neuron pattern of aberrant motor unit potential firing rates. These findings are consistent with dysfunction of the corticospinal pathways rather than a lower motor unit." Significant low extremity weakness.',
       'spasticity/stiff legs', 'spastic diplegic cerebral palsy',
       'orobuccal dyspraxia, awkward gross and fine motricity, difficulty in coordination, unstable gait'],
      dtype=object)

In [7]:
type(df['neurological examination'])

pandas.core.series.Series

In [12]:
neuro_exam_custom_map = {'low extremity weakness': 'HP:0007340',  # Lower limb muscle weakness HP:0007340
                         'unstable gait': 'HP:0002317' # Unsteady gait HP:0002317
                        }

In [13]:
neuroMapper = ColumnMapper(custom_map_d=neuro_exam_custom_map, concept_recognizer = hpo_cr)


In [14]:
neuroMapper.preview_column(df['neurological examination'])

parse minichunks for ['nan']
parse candidate for 'nan'
parse minichunks for ['nan']
parse candidate for 'nan'
parse minichunks for ['nan']
parse candidate for 'nan'
parse minichunks for ['nan']
parse candidate for 'nan'
parse minichunks for ['nan']
parse candidate for 'nan'
parse minichunks for ['spasticity', ' nerve conduction and emg studies with abnormal findings "remarkable for the failure to activate the leg muscles due to an upper motor neuron pattern of aberrant motor unit potential firing rates. these findings are consistent with dysfunction of the corticospinal pathways rather than a lower motor unit." significant low extremity weakness.']
parse candidate for 'spasticity'
candidate 'spasticity' in self._label_to_id
parse candidate for 'nerve conduction and emg studies with abnormal findings "remarkable for the failure to activate the leg muscles due to an upper motor neuron pattern of aberrant motor unit potential firing rates. these findings are consistent with dysfunction of

,column,terms
0,ataxia,Ataxia (HP:0001251)
1,NaN,n/a
2,spastic paraplegia,Spastic paraplegia (HP:0001258)
3,"spasticity; nerve conduction and EMG studies with abnormal findings ""remarkable for the failure to activate the leg muscles due to an upper motor neuron pattern of aberrant motor unit potential firing rates. These findings are consistent with dysfunction of the corticospinal pathways rather than a lower motor unit."" Significant low extremity weakness.",Spasticity (HP:0001257)
4,spasticity/stiff legs,Spasticity (HP:0001257)
5,spastic diplegic cerebral palsy,n/a
6,"orobuccal dyspraxia, awkward gross and fine motricity, difficulty in coordination, unstable gait",Unsteady gait (HP:0002317)


In [11]:
cohort_encoder = CohortEncoder(df=df, hpo_cr=hpo_cr)